![AB Testing](https://www.weidert.com/hs-fs/hubfs/blog_images/AB%20Testing%20best%20practices.jpeg?width=1800&height=945&name=AB%20Testing%20best%20practices.jpeg)

# WHAT IS THE AB TESTING ?

A/B testing, also known as split testing, is a method of comparing two variants(A and B) to determine which one performs better. In A/B testing, two variants (A and B) are created with one element of the design or content being different between the two variants. The two versions are then shown randomly to users, and their behavior is monitored and compared to determine which variant performs better based on a predefined goal, such as the conversion rate, click-through rate, or engagement rate.

A/B testing is commonly used in website optimization, digital marketing, and product development to determine which variant is more effective at achieving a specific goal. It helps businesses make data-driven decisions by providing insight into what resonates better with their audience and what changes can be made to improve the user experience and increase conversions.

# Hypothesis Testing

Hypothesis tests are analytical tools used as a statistical method. These tests aim to test a hypothesis (claim) about a population using sample data. Hypothesis tests help us make inferences about whether a hypothesis is true or not by collecting statistical evidence.

Generally, two hypotheses are formulated: the null hypothesis (H0) and the alternative hypothesis (H1 or HA). The null hypothesis represents the existing condition or the accepted assumptions, while the alternative hypothesis claims a specific change or effect.

Hypothesis tests can be categorized into the following types:

- **One-Sample T Test:** Used to test if the mean of a population is significantly different from a specific value.

- **Independent Two-Sample T Test:** Used to test if there is a significant difference between the means of two independent groups.

- **Paired Two-Sample T Test:** Used to test if there is a significant difference between the means of two related or paired groups.

- **Analysis of Variance (ANOVA):** Used to test if there is a significant difference among the means of three or more groups.

- **Chi-Square Test:** Used to test if there is a dependency or association between two categorical variables.

- **Regression Analysis:** Used to analyze the relationship between dependent and independent variables.

These are just some popular types of hypothesis tests, and there are many more methods available in statistics. Hypothesis tests are used in various fields, ranging from scientific research and marketing strategies to medical studies and industrial quality control processes. These tests enable us to make objective decisions based on data and evaluate statistical significance.

- Hypothesis tests are statistical methods used to test a belief or proposition.

- Within the scope of hypothesis testing, there are group comparisons.

- The main objective in group comparisons is to determine whether observed differences are due to chance or if there is a genuine difference.

**For example:**

- Did the average daily usage time of users increase after a user interface change in a mobile application?

- The result we obtain from this analysis, based on the sample we have, will help us determine if the observed outcome occurred by chance or if there is indeed a significant difference.

- We will strive to understand this through statistical calculations and analysis. We will provide evidence to support our findings.

# A/B Testing (Independent Two-Sample T Test) (Comparing Two Group Means)

# Road Map
1. Set up Hypotheses
2. Assumption Check
    - 2.1. Assumption of Normality
    - 2.2. Variance Homogeneity
3. Implementation of the Hypothesis
    - 3.1. If the assumptions are met, independent two sample t test (parametric test)
    - 3.2.If assumptions are not met, mannwhitneyu test (non-parametric test)
4. Interpret results according to p-value

In [1]:
# import Required Libraries

import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu,pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

In [2]:
# Adjusting Row Column Settings

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Exercise 3: Is There a Statistically Significant Difference Between the Mean Age of People with and Without Diabetes?

In [3]:
# Loading the Data Set

df = pd.read_csv("/kaggle/input/docspot/datasets_228_482_diabetes.csv")

In [4]:
# Preliminary examination of the data set

def check_df(dataframe, head=5):
    print('##################### Shape #####################')
    print(dataframe.shape)
    print('##################### Types #####################')
    print(dataframe.dtypes)
    print('##################### Head #####################')
    print(dataframe.head(head))
    print('##################### Tail #####################')
    print(dataframe.tail(head))
    print('##################### NA #####################')
    print(dataframe.isnull().sum())
    print('##################### Quantiles #####################')
    print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

##################### Shape #####################
(768, 9)
##################### Types #####################
Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object
##################### Head #####################
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin      BMI  \
0            6      148             72             35        0 33.60000   
1            1       85             66             29        0 26.60000   
2            8      183             64              0        0 23.30000   
3            1       89             66             23       94 28.10000   
4            0      137             40             35      168 43.10000   

   DiabetesPedigreeFunction  Age  Outcome  
0        

In [5]:
df.groupby("Outcome").agg({"Age": "mean"})

,Age
Outcome,
0,31.19000
1,37.06716


In [6]:
# There seems to be a mathematical difference between the two groups. But is this difference a chance occurrence or is it statistically significant?

# 1. Establish the hypotheses:

- **H0: M1 = M2** (There is no statistically significant difference between the mean ages of those with and without diabetes)
- **H1: M1! = M2** (There is a statistically significant difference between the mean ages of those with and without diabetes)

# 2. Examine Assumptions

**Assumption of normality**

- H0: The assumption of normal distribution is satisfied.
- H1: Assumption of normal distribution is not satisfied

In [7]:
# H0 REJ if p-value < 0.05.
# H0 CANNOT BE REJECTED if p-value < 0.05.

In [8]:
test_stat, pvalue = shapiro(df.loc[df["Outcome"] == 1, "Age"].dropna())

In [9]:
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9546, p-value = 0.0000


In [10]:
test_stat, pvalue = shapiro(df.loc[df["Outcome"] == 0, "Age"].dropna())

In [11]:
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.8012, p-value = 0.0000


In [12]:
# Since the p-value is less than 0.05, hypothesis H0 is rejected.
# The assumption of normality is not met.

- **Non-parametric test is applied because normality assumption is not satisfied. mannwhitneyu test is applied.**
- **Since the assumption of normality is not satisfied, we do not need to perform the homogeneity of variance test.**

In [13]:
# Hipotez (H0: M1 = M2)

In [14]:
test_stat, pvalue = mannwhitneyu(df.loc[df["Outcome"] == 1, "Age"].dropna(),
                                 df.loc[df["Outcome"] == 0, "Age"].dropna())

In [15]:
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 92050.0000, p-value = 0.0000


In [16]:
# H0 hypothesis is rejected.
# The hypothesis that there is no statistically significant difference between the mean ages of people with and without diabetes is rejected.
# There is a statistically significant difference between the mean ages of those with and without diabetes.

# In other words, those with higher age have diabetes.
